In [2]:
import os
BUCKET = 
os.environ["BUCKET"] = BUCKET



In [14]:
import os, requests
from pathlib import Path


OUT = Path("../data/raw"); OUT.mkdir(parents=True, exist_ok=True)
urls = [
    "https://faolex.fao.org/docs/pdf/ssd127441.pdf",
    "https://www.eia.gov/international/content/analysis/countries_long/Sudan_and_South_Sudan/pdf/Sudans%20CAB%20FY2024.pdf",
    "https://verite.org/wp-content/uploads/2021/07/Verite-Country-Report-South-Sudan.pdf",
    "https://www.usip.org/sites/default/files/2021-04/sr_493-conflict_and_crisis_in_south_sudans_equatoria.pdf",
    "https://libraries.indiana.edu/libraries/pdf.js/web/viewer.html?file=https%3A%2F%2Flibraries.indiana.edu%2Fsites%2Fdefault%2Ffiles%2FSOUTH%2520SUDAN.pdf",
    "https://www.u4.no/publications/south-sudan-overview-of-corruption-and-anti-corruption.pdf",
    "https://documents1.worldbank.org/curated/en/099031825031520159/pdf/P500556-9f568594-b30c-4fcc-bb7a-b96df92ed5fd.pdf",
    "https://www.state.gov/wp-content/uploads/2021/03/SOUTH-SUDAN-2020-HUMAN-RIGHTS-REPORT.pdf",
    "https://assets.publishing.service.gov.uk/media/671267b5b40d67191077b398/South_Sudan_Toponymic_Factfile.pdf",
    "https://bti-project.org/fileadmin/api/content/en/downloads/reports/country_report_2022_SSD.pdf",
    "https://era.ed.ac.uk/bitstream/handle/1842/42410/Perceiving-Peace-in-a-Fragment-State-The-Case-of-South-Sudan-DIGITAL.pdf?sequence=1&isAllowed=y",
    "https://www.brookings.edu/wp-content/uploads/2016/06/06-south-sudan.pdf",
    "https://www.cfr.org/sites/default/files/pdf/2016/11/CSR77_Knopf_South%20Sudan.pdf",
    "https://www.ssoar.info/ssoar/bitstream/handle/document/67602/ssoar-jlibertyintaff-2020-1-afriyie_et_al-Comprehensive_analysis_of_South_Sudan.pdf",
    "https://journalistsresource.org/wp-content/uploads/2011/08/South-Sudan.pdf",
    "https://ciaotest.cc.columbia.edu/journals/ambrev/ambrev1257/f_0029920_24220.pdf",
    "https://sgp.fas.org/crs/row/R43344.pdf",
    "https://docs.pca-cpa.org/2016/02/South-Sudan-Peace-Agreement-September-2018.pdf",
    "https://ucdpged.uu.se/peaceagreements/fulltext/SD_120927_Cooperation%20Agreement%20between%20Sudan%20and%20South%20Sudan.pdf",
    "https://www.rahs-open-lid.com/wp-content/uploads/2024/02/South-Sudan_-The-Untold-Story-from-Independence-to-Civil-War-PDFDrive-.pdf",
    "https://carnegie-production-assets.s3.amazonaws.com/static/files/files__sudan_conflict.pdf",
    "https://www.impact-se.org/wp-content/uploads/South-Sudan.pdf",
    "https://eprints.lse.ac.uk/108888/1/McCrone_the_wars_in_South_Sudan_published.pdf",
    "https://med.virginia.edu/family-medicine/wp-content/uploads/sites/285/2018/12/Azobou_South-Sudan-Refugee-Crisis-112018.pdf",
    "https://riftvalley.net/wp-content/uploads/2018/06/RVI-The-Sudan-Handbook.pdf",
    "https://www.congress.gov/crs_external_products/IF/PDF/IF10218/IF10218.14.pdf",
    "https://docs.southsudanngoforum.org/sites/default/files/2018-01/Labour%20Act%202017.pdf",
    "https://mojca.gov.ss/wp-content/uploads/2023/03/Registration-of-Business-Names-Act-7-of-2008.pdf",
    "https://archive.doingbusiness.org/content/dam/doingBusiness/country/s/south-sudan/SSD.pdf",
    "https://www.wto.org/english/thewto_e/acc_e/ssd_e/wtaccssd6_leg_1.pdf",
    "https://mojca.gov.ss/wp-content/uploads/2023/03/Interim-Constitution-of-Southern-Sudan-2005.pdf",
    "https://faolex.fao.org/docs/pdf/ssd127441.pdf",
    "https://www.refworld.org/sites/default/files/attachments/531469ee6.pdf",
    "https://www.homeaffairs.gov.au/foi/files/2021/fa-210700890-document-released.PDF",
    "https://togetherwomenrise.org/wp-content/uploads/2018/08/SouthSudan.pdf",
    "https://faolex.fao.org/docs/pdf/ssd199925.pdf",
    "https://mofaic.gov.ss/wp-content/uploads/2023/03/Investment-Promotion-Act-2009.pdf",
]
for i,u in enumerate(urls,1):
    p = OUT / f"doc_{i:03}.pdf"
    with requests.get(u, timeout=60) as r:
        r.raise_for_status()
        p.write_bytes(r.content)
print("complete")

complete


In [17]:

import pathlib
from pdfminer.high_level import extract_text as pdf_text
from readability import Document
from bs4 import BeautifulSoup

INP = pathlib.Path("../data/raw")
OUT = pathlib.Path("../data/interim")
OUT.mkdir(parents=True, exist_ok=True)

for p in INP.iterdir():
    if p.suffix.lower() == ".pdf":
        try:
            txt = pdf_text(open(p, 'rb'))
        except Exception as e:
            print(f"Skipping {p.name}: not a valid PDF ({e})")
            continue
    else:
        html = p.read_text(encoding='utf-8', errors='ignore')
        main = Document(html).summary()
        txt = BeautifulSoup(main, 'html.parser').get_text("")
    (OUT / f"{p.stem}.txt").write_text(txt, encoding='utf-8')

Skipping doc_005.pdf: not a valid PDF (No /Root object! - Is this really a PDF?)


Skipping doc_008.pdf: not a valid PDF (No /Root object! - Is this really a PDF?)


In [18]:
#chunk
import json, uuid, re
from pathlib import Path

INP = Path("../data/interim")
OUT = Path("../data/processed")
OUT.mkdir(parents=True, exist_ok=True)
CHUNK, OVERLAP = 800, 120

for p in INP.glob("*.txt"):
    try:
        t = re.sub(r"\n{3,}", "\n\n", p.read_text(encoding='utf-8')).strip()
        i, parts = 0, []
        while i < len(t):
            parts.append(t[i:i+CHUNK])
            i += CHUNK - OVERLAP
        with open(OUT / f"{p.stem}.jsonl", 'w', encoding='utf-8') as f:
            for k, c in enumerate(parts):
                try:
                    rec = {
                        "chunk_id": str(uuid.uuid4()),
                        "source": p.stem,
                        "order": k,
                        "text": c
                    }
                    f.write(json.dumps(rec) + "\n")
                except Exception as e:
                    print(f"Error writing chunk {k} of {p.name}: {e}")
    except Exception as e:
        print(f"Error processing {p.name}: {e}")                    
                    

In [20]:
#enrich       
import spacy, json
from pathlib import Path

nlp = spacy.load("en_core_web_sm")
for jf in Path("../data/processed").glob("*.jsonl"):
    try:
        lines = [json.loads(x) for x in open(jf, encoding='utf-8')]
        for r in lines:
            try:
                doc = nlp(r["text"])
                r["entities"] = [(e.text, e.label_) for e in doc.ents]
            except Exception as e:
                print(f"NER error in {jf.name} chunk {r.get('order', '?')}: {e}")
                r["entities"] = []
        with open(jf, 'w', encoding='utf-8') as f:
            f.write("\n".join(json.dumps(r) for r in lines))
    except Exception as e:
        print(f"Error processing {jf.name}: {e}")                                        

In [21]:
# Embed & FAISS index
import faiss, numpy as np, json
from pathlib import Path
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
chunks, metas = [], []

for jf in Path("../data/processed").glob("*.jsonl"):
    try:
        with open(jf, encoding='utf-8') as f:
            for line in f:
                try:
                    rec = json.loads(line)
                    chunks.append(rec['text'])
                    metas.append({k: rec[k] for k in ("chunk_id", "source", "order")})
                except Exception as e:
                    print(f"Error parsing line in {jf.name}: {e}")
    except Exception as e:
        print(f"Error opening {jf.name}: {e}")

try:
    X = model.encode(chunks, batch_size=64, normalize_embeddings=True).astype('float32')
except Exception as e:
    print(f"Error during embedding: {e}")
    X = np.zeros((len(chunks), 384), dtype='float32')  # fallback shape

try:
    index = faiss.IndexFlatIP(X.shape[1])
    index.add(X)
    Path("../data/index").mkdir(parents=True, exist_ok=True)
    faiss.write_index(index, "../data/index/faiss.index")
    json.dump({"chunks": chunks}, open("../data/index/chunks.json", "w", encoding='utf-8'))
    json.dump({"metas": metas}, open("../data/index/meta.json", "w", encoding='utf-8'))
except Exception as e:
    print(f"Error during FAISS indexing or saving: {e}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#search helper
import faiss, numpy as np, json
from sentence_transformers import SentenceTransformer

try:
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
except Exception as e:
    raise RuntimeError(f"Error loading embedding model: {e}")

try:
    index = faiss.read_index("../data/index/faiss.index")
except Exception as e:
    raise RuntimeError(f"Error loading FAISS index: {e}")

try:
    with open("../data/index/chunks.json", encoding="utf-8") as f:
        chunks = json.load(f)['chunks']
    with open("../data/index/meta.json", encoding="utf-8") as f:
        metas = json.load(f)['metas']
except Exception as e:
    raise RuntimeError(f"Error loading chunk/meta files: {e}")

def topk(query, k=5):
    try:
        qv = model.encode([query], normalize_embeddings=True).astype('float32')
        D, I = index.search(qv, k)
        return [{"score": float(D[0][j]), "text": chunks[i], "meta": metas[i]} for j, i in enumerate(I[0])]
    except Exception as e:
        print(f"Error during search: {e}")
        return []


In [8]:
# Prompt builder
SYSTEM = (
    "Answer using ONLY the provided context. Cite sources inline like [1], [2]. "
    "If not in context, say you cannot find it."
)

def build_prompt(q, hits):
        ctx = []
        for i, h in enumerate(hits, 1):
                try:
                        src = f"{h['meta']['source']}#chunk{h['meta']['order']}"
                        text = h.get('text', '')
                        ctx.append(f"[{i}] {src}:\n{text}")
                except Exception as e:
                        print(f"Error building context for hit {i}: {e}")
                        continue
        try:
                prompt = SYSTEM + "\n\n" + "\n\n".join(ctx) + f"\n\nQuestion: {q}\nAnswer with citations."
                return prompt
        except Exception as e:
                print(f"Error building prompt: {e}")
                return SYSTEM + f"\n\nQuestion: {q}\nAnswer with citations."

In [9]:
#LLM call 

from openai import OpenAI, APIError
import os

client = OpenAI()

def answer(q, k=5):
    try:
        hits = topk(q, k)
        prompt = build_prompt(q, hits)
        resp = client.chat.completions.create(
            model="gpt-4o-mini", temperature=0.2,
            messages=[
                {"role": "system", "content": "You are a careful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return resp.choices[0].message.content, hits
    except APIError as e:
        print(f"OpenAI API error: {e}")
        return "Sorry, there was an API error.", []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "Sorry, an unexpected error occurred.", []

if __name__ == "__main__":
    while True:
        try:
            q = input("Enter your question (or 'exit' to quit): ")
            if q.strip().lower() == "exit":
                break
            answer_text, hits = answer(q)
            print("\nAnswer:\n", answer_text)
            print("\nSources:")
            for i, h in enumerate(hits, 1):
                print(f"[{i}] {h['meta']['source']}#chunk{h['meta']['order']}")
            print("-" * 40)
        except KeyboardInterrupt:
            print("\nExiting.")
            break

Enter your question (or 'exit' to quit):  When did south sudan gain independence



Answer:
 South Sudan gained independence on July 9, 2011, following a referendum held in January 2011, where 98.83% of the voters supported independence [2], [5].

Sources:
[1] doc_012#chunk5
[2] doc_010#chunk8
[3] doc_025#chunk25
[4] doc_016#chunk4
[5] doc_024#chunk3
----------------------------------------

Exiting.
